# Task 2: Char-RNN

Char-RNN implements multi-layer Recurrent Neural Network (RNN, LSTM, and GRU) for training/sampling from character-level language models. In other words the model takes one text file as input and trains a Recurrent Neural Network that learns to predict the next character in a sequence. The RNN can then be used to generate text character by character that will look like the original training data. This network is first posted by Andrej Karpathy, you can find out about his original code on https://github.com/karpathy/char-rnn, the original code is written in *lua*.

Here we will implement Char-RNN using Tensorflow!

In [2]:
import time
import numpy as np
import tensorflow as tf

# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

## Part 1: Setup
In this part, we will read the data of our input text and process the text for later network training. There are two txt files in the data folder, for computing time consideration, we will use tinyshakespeare.txt here.

In [3]:
with open('data/tinyshakespeare.txt', 'r') as f:
    text=f.read()
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))
# and let's get a glance of what the text is
print(text[:500])

Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [5]:
# Creating a mapping from unique characters to indices
vocab_to_ind = {c: i for i, c in enumerate(vocab)}
ind_to_vocab = dict(enumerate(vocab))
text_as_int = np.array([vocab_to_ind[c] for c in text], dtype=np.int32)

# We mapped the character as indexes from 0 to len(vocab)
for char,_ in zip(vocab_to_ind, range(20)):
    print('{:6s} ---> {:4d}'.format(repr(char), vocab_to_ind[char]))
# Show how the first 10 characters from the text are mapped to integers
print ('{} --- characters mapped to int --- > {}'.format(text[:10], text_as_int[:10]))

'\n'   --->    0
' '    --->    1
'!'    --->    2
'$'    --->    3
'&'    --->    4
"'"    --->    5
','    --->    6
'-'    --->    7
'.'    --->    8
'3'    --->    9
':'    --->   10
';'    --->   11
'?'    --->   12
'A'    --->   13
'B'    --->   14
'C'    --->   15
'D'    --->   16
'E'    --->   17
'F'    --->   18
'G'    --->   19
First Citi --- characters mapped to int --- > [18 47 56 57 58  1 15 47 58 47]


## Part 2: Creating batches
Now that we have preprocessed our input data, we then need to partition our data, here we will use mini-batches to train our model, so how will we define our batches?

Let's first clarify the concepts of batches:
1. **batch_size**: Reviewing batches in CNN, if we have 100 samples and we set batch_size as 10, it means that we will send 10 samples to the network at one time. In RNN, batch_size have the same meaning, it defines how many samples we send to the network at one time.
2. **sequence_length**: However, as for RNN, we store memory in our cells, we pass the information through cells, so we have this sequence_length concept, which also called 'steps', it defines how long a sequence is.

From above two concepts, we here clarify the meaning of batch_size in RNN. Here, we define the number of sequences in a batch as N and the length of each sequence as M, so batch_size in RNN **still** represent the number of sequences in a batch but the data size of a batch is actually an array of size **[N, M]**.

<span style="color:red">TODO:</span>
finish the get_batches() function below to generate mini-batches.

Hint: this function defines a generator, use *yield*.

In [6]:
def get_batches(array, n_seqs, n_steps):
    '''
    Partition data array into mini-batches
    input:
    array: input data
    n_seqs: number of sequences in a batch
    n_steps: length of each sequence
    output:
    x: inputs
    y: targets, which is x with one position shift
       you can check the following figure to get the sence of what a target looks like
    '''
    batch_size = n_seqs * n_steps
    n_batches = int(len(array) / batch_size)
    # we only keep the full batches and ignore the left.
    array = array[:batch_size * n_batches]
    array = array.reshape((n_seqs, -1))
    
    # You should now create a loop to generate batches for inputs and targets
    #############################################
    #           TODO: YOUR CODE HERE            #
    #############################################
    
    for i in range(n_seqs, array.shape[1]):
        yield array[:,i-n_seqs:i], array[:,i-n_seqs+1:i+1]

In [7]:
batches = get_batches(text_as_int, 10, 10)
x, y = next(batches)
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[18 47 56 57 58  1 15 47 58 47]
 [ 1 43 52 43 51 63 11  0 37 43]
 [52 58 43 42  1 60 47 56 58 59]
 [56 44 53 50 49  6  0 27 52  1]
 [47 52  1 57 54 47 58 43  1 53]
 [56 57  6  1 39 52 42  1 57 58]
 [46 47 51  1 42 53 61 52  1 58]
 [ 1 40 43 43 52  1 57 47 52 41]
 [50 58 57  1 51 39 63  1 57 46]
 [57 47 53 52  1 53 44  1 56 43]]

y
 [[47 56 57 58  1 15 47 58 47 64]
 [43 52 43 51 63 11  0 37 43 58]
 [58 43 42  1 60 47 56 58 59 43]
 [44 53 50 49  6  0 27 52  1 54]
 [52  1 57 54 47 58 43  1 53 44]
 [57  6  1 39 52 42  1 57 58 39]
 [47 51  1 42 53 61 52  1 58 53]
 [40 43 43 52  1 57 47 52 41 43]
 [58 57  1 51 39 63  1 57 46 39]
 [47 53 52  1 53 44  1 56 43 60]]


## Part 3: Build Char-RNN model
In this section, we will build our char-rnn model, it consists of input layer, rnn_cell layer, output layer, loss and optimizer, we will build them one by one.

The goal is to predict new text after given prime word, so for our training data, we have to define inputs and targets, here is a figure that explains the structure of the Char-RNN network.

![structure](img/charrnn.jpg)

<span style="color:red">TODO:</span>
finish all TODOs in ecbm4040.CharRNN and the blanks in the following cells.

**Note: The training process on following settings of parameters takes about 20 minutes on a GTX 1070 GPU, so you are suggested to use GCP for this task.**

In [9]:
from ecbm4040.CharRNN import *

### Training
Set sampling as False(default), we can start training the network, we automatically save checkpoints in the folder /checkpoints.

In [8]:
# these are preset parameters, you can change them to get better result
batch_size = 100         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
rnn_size = 256           # Size of hidden layers in rnn_cell
num_layers = 2           # Number of hidden layers
learning_rate = 0.005    # Learning rate

In [10]:
model = CharRNN(len(vocab), batch_size, num_steps, 'LSTM', rnn_size,
               num_layers, learning_rate)
batches = get_batches(text_as_int, batch_size, num_steps)
model.train(batches, 6000, 2000)

step: 200  loss: 2.6566  0.2294 sec/batch
step: 400  loss: 2.4740  0.2293 sec/batch
step: 600  loss: 2.3747  0.2334 sec/batch
step: 800  loss: 2.3997  0.2297 sec/batch
step: 1000  loss: 2.3928  0.2271 sec/batch
step: 1200  loss: 2.3154  0.2321 sec/batch
step: 1400  loss: 2.3491  0.2358 sec/batch
step: 1600  loss: 2.3216  0.2255 sec/batch
step: 1800  loss: 2.3367  0.2323 sec/batch
step: 2000  loss: 2.2879  0.2300 sec/batch
step: 2200  loss: 2.3515  0.2291 sec/batch
step: 2400  loss: 2.2853  0.2321 sec/batch
step: 2600  loss: 2.3126  0.2304 sec/batch
step: 2800  loss: 2.2776  0.2283 sec/batch
step: 3000  loss: 2.2754  0.2298 sec/batch
step: 3200  loss: 2.2735  0.2293 sec/batch
step: 3400  loss: 2.2893  0.2315 sec/batch
step: 3600  loss: 2.2674  0.2310 sec/batch
step: 3800  loss: 2.2372  0.2315 sec/batch
step: 4000  loss: 2.2490  0.2281 sec/batch
step: 4200  loss: 2.2841  0.2288 sec/batch
step: 4400  loss: 2.2493  0.2265 sec/batch
step: 4600  loss: 2.2734  0.2328 sec/batch
step: 4800  los

In [17]:
# look up checkpoints
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i6000_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i2000_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i4000_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i6000_l256.ckpt"

### Sampling
Set the sampling as True and we can generate new characters one by one. We can use our saved checkpoints to see how the network learned gradually.

In [28]:
model = CharRNN(len(vocab), batch_size, num_steps, 'LSTM',rnn_size,
               num_layers, learning_rate, sampling=True)
# choose the last checkpoint and generate new text
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = model.sample(checkpoint, 1000, len(vocab), vocab_to_ind, ind_to_vocab, prime="LORD ")
print(samp)
print(''.join(samp))


INFO:tensorflow:Restoring parameters from checkpoints/i6000_l256.ckpt
['L' 'O' 'R' ... ' ' 't' ' ']
LORD h eae ee there fas o  our h ayoreet witetha etheaat tha  ouae m h an tt  aresthi oveet w t  y  met t h his ane a w tht ae thehentand is
A wt  h mythe  manchther a sor ter ar e aooeraaouh ou t onouofe at th toteou tr fa  e or he myo he s hesooth  t no eshano tte e ouoeon w ad we he  oe tha he en hnoa l hinoourimor   atea ae anoot t  a he  ot mo th
The w hoeaat h amad  hti s, irotae me   ooo  t tht   or t, aaendtaooo t  t w t  hot hanoat oe t w ho ad hon hia he wis  owi adarou a the erethethaant  ade maofo  e  t  te  melle thesoneeree oto  totero et te thor ht 
ae  therour tof wit ht   s aal t ft or o we atee a  t teeo ome faan w eou  w ed wa ou se then  tinthaed  tt   tht wot e t  aroeaee ar    thoe e we han aret w  ane o t te w  eof t ondee m ofehe se   t ote
ou antitont ee ot at tha e w lto oue  sh  tt atoeea   te wot hothaod atoe we t al ane a aoe anoan out  hethe he ads athe  wal

In [29]:
# choose a checkpoint other than the final one and see the results. It could be nasty, don't worry!
#############################################
#           TODO: YOUR CODE HERE            #
#############################################
samp = model.sample('checkpoints/i2000_l256.ckpt', 1000, len(vocab), vocab_to_ind, ind_to_vocab, prime="LORD ")
print(''.join(samp))

INFO:tensorflow:Restoring parameters from checkpoints/i2000_l256.ckpt
LORD ono nov ousthre
Weth outth eo   he  se a he  h teend st   t t t sothiowhthe hila  tane s the aoe   thithee  t reenor h w h sorienast t atroth
touthe
O:
 e t byt eth t, t,
TUC e ereeer e boo oon
 thttoeteon haeen ae wakof ws wat s anee ath touselhout herori en t oar,
eea  and b te stotteh  te aheee st tt we   we w s te enhast arious    e ae weroer endhoenara
oe s th te athho teroust hae  w  ous  oothaenoenher st se ere hasouseooongen  are s ha thah
 eoe  eleoustheel hou hers be stheso th t  stht t   o a  at a sthtal sattoeoat s hoot s eser sou otohars, t t athint s t se es wit s eeheeous
  tthis anesoe, t t are sthofo s t hah  eoutoof e te ano wthis a otte t an fon  hay, ore s h weouoeot s shite o thack st ang ou
t o tar teneeesattoun snt e want toushehitith e thi noonoeht encoale e   e haneer  b tont  t  s, ao t a and tt  o  moth ttrace e tonee  o heareotehorow e oour  wnd outhe ant  s  e e o s ssor haayoou s  s

### Change another type of RNN cell
We are using LSTM cell as the original work, but GRU cell is getting more popular today, let's chage the cell in rnn_cell layer to GRU cell and see how it performs. Your number of step should be the same as above.

**Note: You need to change your saved checkpoints' name or they will rewrite the LSTM results that you have already saved.**

In [10]:
# these are preset parameters, you can change them to get better result
batch_size = 100         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
rnn_size = 256           # Size of hidden layers in rnn_cell
num_layers = 2           # Number of hidden layers
learning_rate = 0.005    # Learning rate

model = CharRNN(len(vocab), batch_size, num_steps, 'GRU', rnn_size,
               num_layers, learning_rate)
batches = get_batches(text_as_int, batch_size, num_steps)
model.train(batches, 6000, 2000)

step: 200  loss: 2.5569  0.2294 sec/batch
step: 400  loss: 2.4059  0.2293 sec/batch
step: 600  loss: 2.3781  0.2288 sec/batch
step: 800  loss: 2.3747  0.2265 sec/batch
step: 1000  loss: 2.3534  0.2214 sec/batch
step: 1200  loss: 2.3154  0.2241 sec/batch
step: 1400  loss: 2.3240  0.2248 sec/batch
step: 1600  loss: 2.3056  0.2338 sec/batch
step: 1800  loss: 2.3119  0.2224 sec/batch
step: 2000  loss: 2.3015  0.2268 sec/batch
step: 2200  loss: 2.3344  0.2284 sec/batch
step: 2400  loss: 2.2753  0.2238 sec/batch
step: 2600  loss: 2.2775  0.2267 sec/batch
step: 2800  loss: 2.2789  0.2348 sec/batch
step: 3000  loss: 2.2831  0.4045 sec/batch
step: 3200  loss: 2.2701  0.2267 sec/batch
step: 3400  loss: 2.2940  0.2288 sec/batch
step: 3600  loss: 2.2339  0.2270 sec/batch
step: 3800  loss: 2.2710  0.2231 sec/batch
step: 4000  loss: 2.2475  0.2215 sec/batch
step: 4200  loss: 2.3031  0.2265 sec/batch
step: 4400  loss: 2.3123  0.2267 sec/batch
step: 4600  loss: 2.2946  0.2288 sec/batch
step: 4800  los

In [12]:
model = CharRNN(len(vocab), batch_size, num_steps, 'GRU', rnn_size,
               num_layers, learning_rate, sampling=True)
# choose the last checkpoint and generate new text
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = model.sample(checkpoint, 1000, len(vocab), vocab_to_ind, ind_to_vocab, prime="LORD ")
print(samp)
print(''.join(samp))

INFO:tensorflow:Restoring parameters from checkpoints/i6000_l256.ckpt
['L' 'O' 'R' ... ' ' 's' 't']
LORD  hat tae e that aast aehid hes attis men st t wed  s wt sheo an m ou e eent ofat a ha  sat et a tarir  a at  th to a antel  taath iaeeathare  hoto   h a     allee  of  thaoert hohean owt o  toat a to o a ohat aatot   t t th o sehite   a ttoea t hetti t woh thohh ale hee aantoalin   t atitethae weriof t t  h   oo t o hede t ast m   at th   esee mese ae  sar  th w t tho artte t t ath ter hhere tat s hot tnthtetto ot  sheo  er htoorett etee o t e alo thite  t  an ta tththo at e waheto  aethe s  t an atetheert, edthat  anathe tth as m he t  n ssteonoe es  o ho our thitatiout tet es teou t alatht t me i tere at  heeee hte h  ttee w enous t t s t s t  hihaoto    at t sthe moo artha  thoe tieete athe s tenof eat e  terote we t ahe h t   o o h t wha ttes housera oesowo e tt  thalie watt t enofa t  h ert ws tt tthhare  seas as ttree aed taneneo eno o  hottho oe tet tomeonhae   ee tone hes  o

#### Questions
1. Compare your result of two networks that you built and the reasons that caused the difference. (It is a qualitative comparison, it should be based on the specific model that you build.)
2. Discuss the difference between LSTM cells and GRU cells, what are the pros and cons of using GRU cells?

Answer:
Ans 1: Both the models are outputting garbage and need to be trained better. But GRU model kept on repeating few characters very frequently and LSTM seemed to have better potential to get trained better.

Ans 2 : (Source: Internet blog)
The GRU unit controls the flow of information like the LSTM unit, but without having to use a memory unit. It just exposes the full hidden content without any control. GRU is computationally more efficient because of the less complex structure. But LSTMs remember longer sequences than GRUs. LSTM would outperform it in tasks requiring modeling long-distance relations.